In [ ]:
!pip install -q -U google-generativeai

In [ ]:
# Download video file
!wget https://storage.googleapis.com/generativeai-downloads/data/Sherlock_Jr_FullMovie.mp4

In [ ]:
from google.generativeai import caching
import datetime
import time

In [ ]:
from google.colab import output
from IPython.display import display, Markdown
from IPython.display import Image
import google.generativeai as genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY= userdata.get("GOOGLE_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
video_file_name = "Sherlock_Jr_FullMovie.mp4"

# Upload the video using the Files API
video_file = genai.upload_file(path=video_file_name)

In [ ]:
# Wait for the file to finish processing
while video_file.state.name == "PROCESSING":
    print("Waiting for video to be processed.")
    time.sleep(2)
    video_file = genai.get_file(video_file.name)

print(f"Video processing complete: {video_file.uri}")

In [ ]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-001")

In [ ]:
%%time
normal_call = model.generate_content([video_file, "Tell me about the video and its themes"])

In [ ]:
Markdown(normal_call.text)

In [ ]:
print(normal_call.usage_metadata)

In [ ]:
# Create a cache with a 10 minute TTL
cache = caching.CachedContent.create(
    model="models/gemini-1.5-flash-001",
    display_name="sherlock jr movie", # used to identify the cache
    system_instruction="You are an expert video analyzer, and your job is to answer user's query based on the video file you have access to.",
    contents=[video_file],
    ttl=datetime.timedelta(minutes=10)
)

In [ ]:
cache.expire_time

In [ ]:
%%time

# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)

# Query the model
response = model.generate_content(["Introduce different characters in the movie by describing their personality, looks, and names. Also list the timestamps they were introduced for the first time."])

print(response.usage_metadata)

In [ ]:
Markdown(response.text)

In [ ]:
%%time
# Query the model
response = model.generate_content(["What is the last time Buster Keaton appears in the movie?"])

print(response.usage_metadata)

In [ ]:
Markdown(response.text)